<a href="https://colab.research.google.com/github/margeumkim/BRIDGEMAIL/blob/master/BERT_Evaluation_Vector_Variances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Loading Pre-Trained BERT


Install the pytorch interface for BERT by Hugging Face. (This library contains interfaces for other pretrained language models like OpenAI's GPT and GPT-2.)

We've selected the pytorch interface because it strikes a nice balance between the high-level APIs (which are easy to use but don't provide insight into how things work) and tensorflow code (which contains lots of details but often sidetracks us into lessons about tensorflow, when the purpose here is BERT!).

If you're running this code on Google Colab, you will have to install this library each time you reconnect; the following cell will take care of that for you.

In [1]:
!pip install transformers

     |████████████████████████████████| 675kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 15.1MB/s 
     |████████████████████████████████| 1.1MB 46.7MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a15a327d9bfcaf59de3b1647e287fa39d36f25722d454c783dc886287ac50dbe
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
# Upgrading RAM to 25GB from 12GB
#a = []
#while(1):
#    a.append(‘1’)

Now let's import pytorch, the pretrained BERT model, and a BERT tokenizer.

We'll explain the BERT model in detail in a later tutorial, but this is the pre-trained model released by Google that ran for many, many hours on Wikipedia and Book Corpus, a dataset containing +10,000 books of different genres. This model is responsible (with a little modification) for beating NLP benchmarks across a range of tasks. Google released a few variations of BERT models, but the one we'll use here is the smaller of the two available sizes ("base" and "large") and ignores casing, hence "uncased.""

**transformers** provides a number of classes for applying BERT to different tasks (token classification, text classification, ...). Here, we're using the basic **BertModel** which has no specific output task--it's a good choice for using BERT just to extract embeddings.

In [2]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicCOnfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 2. Loading Enron emails data 
We'll load the Enron Emails dataset. For the purpose of exploratory data analysis, I will subset two accounts -- Kate Symes and Phillip Love. 


Let's import data from my Google drive 
** (but later, will need to combine with git - you must slice file so that each is <25MB) **

> NOTE: When prompted, click on the link to get authentication to allow Google to access your Drive. You should see a screen with “Google Cloud SDK wants to access your Google Account” at the top. After you allow permission, copy the given verification code and paste it in the box in Colab.
Once you have completed verification, go to the CSV file in Google Drive, right-click on it and select “Get shareable link”. The link will be copied into your clipboard. Paste this link into a string variable in Colab.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

path = "/content/drive/My Drive/data/emails.csv"
emails_df = pd.read_csv(path)

# Dataset is now stored in a Pandas Dataframe

In [5]:
# Checking whether the file is loaded properly
print(emails_df.shape)
emails_df.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [7]:
# A single message looks like this
print(emails_df['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


Let's split the information in each message into different fields.

In [0]:
## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [6]:
import email 

# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
del messages

emails_df.head()

,file,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user
0,allen-p/_sent_mail/1.,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",(phillip.allen@enron.com),(tim.belden@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p
1,allen-p/_sent_mail/10.,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",(phillip.allen@enron.com),(john.lavorato@enron.com),Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p
2,allen-p/_sent_mail/100.,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",(phillip.allen@enron.com),(leah.arsdall@enron.com),Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p
3,allen-p/_sent_mail/1000.,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",(phillip.allen@enron.com),(randall.gay@enron.com),,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p
4,allen-p/_sent_mail/1001.,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",(phillip.allen@enron.com),(greg.piper@enron.com),Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p


Now, randomly sample #n emails from the dataset to run BERT and find tokens with contextual meanings

In [0]:
# Let's randomly sample #N sentences 
import random 

sample_size = 100
emails_sample = emails_df.sample(n = sample_size)

Select contents and for each person

In [0]:
content_df = emails_sample["content"].reset_index()
#content_i_df.head()
#content_j_df = account_j_df["content"].reset_index()
#content_j_df.head()

Pre-process text content:


In [0]:
def email_clean(text):
    text = text.rstrip().lower()
    text = " ".join([i for i in text.split() if(not i.isdigit())])
    text = re.sub(r'\n--.*?\n', '', text, flags=re.DOTALL)
    text = re.sub(r'Forwarded by.*?Subject:', '', text, flags=re.DOTALL) 
    text = re.sub(r'Fwd:.*?Subject:', '', text, flags=re.DOTALL) 
    text = re.sub(r'Fw:.*?Subject:', '', text, flags=re.DOTALL)     
    text = re.sub(r'FW:.*?Subject:', '', text, flags=re.DOTALL)         
    text = re.sub(r'Forwarded:.*?Subject:', '', text, flags=re.DOTALL)         
    text = re.sub(r'From:.*?Subject:', '', text, flags=re.DOTALL)
    text = re.sub(r'PM', '', text, flags=re.DOTALL)
    text = re.sub(r'AM', '', text, flags=re.DOTALL)
    text = re.sub(r'enron america corp', '', text, flags=re.DOTALL)
    text = re.sub(r'enron', '', text, flags=re.DOTALL)    
    text = re.sub(r'etc', '', text, flags=re.DOTALL)    
    text = re.sub(r'\n', '', text, flags=re.DOTALL)    
    text = re.sub(r'.*?@.*?', '', text, flags=re.DOTALL)
    text = re.sub(r'http://.*?', '', text, flags=re.DOTALL)    
    text = re.sub(r'cc.*?', '', text, flags=re.DOTALL)    
    text = re.sub(r'subject', '', text, flags=re.DOTALL)    
    text = re.sub(r'from', '', text, flags=re.DOTALL)    
    text = re.sub(r'sent', '', text, flags=re.DOTALL)        
    text = re.sub(r'to', '', text, flags=re.DOTALL)       
    text = re.sub(r'=', '', text, flags=re.DOTALL)            
#    text = re.sub(signoff_1, '', text, flags=re.DOTALL)    
#    text = re.sub(signoff_2, '', text, flags=re.DOTALL)       
#    text = re.sub(signoff_3, '', text, flags=re.DOTALL)   
    return text


def clean(text):
#    stop = set(stopwords.words('english'))
#    stop.update(("to","cc","subject","http","from","sent",
#                 "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa", 
#                 "enron america corp", "enron", "etc", "na", signoff_1, signoff_2, signoff_3))
    exclude = set(string.punctuation) - {'.'}
    lemma = WordNetLemmatizer()
    porter= PorterStemmer()
    
#    text = text.rstrip().lower()
    digit_free = " ".join([i for i in text.lower().split() if(not i.isdigit())])
    punc_free = ''.join(ch for ch in digit_free if ch not in exclude)
#    normalized = " ".join(lemma.lemmatize(word) for word in punc_free)
    #stem = " ".join(porter.stem(token) for token in normalized.split())
    
    return punc_free


In [0]:
import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')

import re
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer 


#signoff_1 = str(account_i_df["user"][0].split('-')[1] + list(account_i_df["user"][0].split('-')[0])[0])
#print(signoff_1)
#signoff_2 = str(list(signoff_1)[0])
#print(signoff_2)
#signoff_3 = str(account_i_sent_df["user"][0].split('-')[0])
#print(signoff_3)

#account_i_clean_content =[]
#for text in account_i_sent_df['content']:
#    email_dot_split = email_clean(text).split('.')
#    for item in email_dot_split:
#        account_i_clean_content.append(item.split('>'))
#    account_i_clean_content.append(clean(email_clean(text)))

#signoff_1 = str(account_j_df["user"][0].split('-')[1] + list(account_j_df["user"][0].split('-')[0])[0])
#print(signoff_1)
#signoff_2 = str(list(signoff_1)[0])
#print(signoff_2)
#signoff_3 = str(account_j_sent_df["user"][0].split('-')[0])
#print(signoff_3)

text_clean_content = []
for text in content_df['content']:
    email_dot_split = email_clean(text).split('.')
    for item in email_dot_split:
        text_clean_content.append(item.split('>'))
#    account_j_clean_content.append(clean(email_clean(text)))

## Flattening nested lists
#account_i_content = [] 
#for text in account_i_clean_content:
#    for item in text:
#      if len(list(item.split())) >3: 
#        if len(list(item.split())) < 101: 
#          account_i_content.append(item)

text_content = [] 
for text in text_clean_content:
    for item in text:
      if len(list(item.split())) >3: 
        if len(list(item.split())) < 101: 
          text_content.append(item)      

In [0]:
# Create labels 
#account_i_content = pd.DataFrame(account_i_content)
#account_i_content.loc[account_i_content.label==0]
#df.loc[df.label == 0].sample(5)[['sentence', 'label']]

In [14]:
print(text_content[0])
len(list(text_content[0].split()))

com] : friday, ocber 19, 10:40 am : anderson, phelps; balling, robert; barthel, scott; bayless, tucker; bear, jon; broadhead, ron; burns, morris; byrom, john; chase, robert; coll, chuck; corbett, john; cox, george; creel, craig; david, edward; debrine, earl; flanagin, mary; french, terri; gallagher, bob; gallegos, gene; garlinger, bob; gilliland, richard; girand, dan; gorham, frank; gratn, patrick; greer, al; guajardo, ronnie; hajny, david; hannifin, patrick; harringn, gerald; hart, korbi; harvard, jeff; hatch, lloyd; heatly, rosemary; hegarty, patrick; henderson, david; hope, sonny; hyatt, kevin; krakauskas, ny; kvasnicka, sally; lee, knute; lee, robert; maw, bill; merrion, t


94

# 3. Tokenization & Input Formatting
Because BERT is a pretrained model that expects input data in a specific format, we will need:

1. A **special token**, [SEP], to mark the end of a sentence, or the separation between two sentences
2. A **special token**, [CLS], at the beginning of our text. This token is used for classification tasks, but BERT expects it no matter what your application is.
3. Tokens that conform with the fixed vocabulary used in BERT
4. The **Token IDs** for the tokens, from BERT's tokenizer
5. **Mask IDs** to indicate which elements in the sequence are tokens and which are padding elements
6. **Segment IDs** used to distinguish different sentences
7. **Positional Embeddings** used to show token position within the sequence
Luckily, the transformers interface takes care of all of the above requirements (using the tokenizer.encode_plus function).

Since this is intended as an introduction to working with BERT, though, we're going to perform these steps in a (mostly) manual way.

For an example of using tokenizer.encode_plus, see the next post on Sentence Classification [here](http://mccormickml.com/2019/07/22/BERT-fine-tuning/).

##3.1. BERT Tokenizer
To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary.

The tokenization must be performed by the tokenizer included with BERT--the below cell will download this for us. We'll be using the "uncased" version here.

In [15]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


Le'ts apply the tokenizer to one sentence just to see the output

In [15]:
# Print the original sentence.
print(' Original: ', content_df['content'][0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(text_content[1]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_content[1])))

 Original:  Kate, this trade has an expiry date of 2001 when it is suppose to be 2003. 
Mike Driskil entered the trade. I have BPA's confirm that also verifies and 
the notes in the bottom of the comment section. Please change if correct.

Thanks,

Kimberly
Tokenized:  ['mike', 'dr', '##isk', '##il', 'entered', 'the', 'trade']
Token IDs:  [3505, 2852, 20573, 4014, 3133, 1996, 3119]


In [16]:
len(text_content)  #2411 sentences

677

When we actually convert all of our sentences, we'll use the tokenize.encode function to handle both steps, rather than calling tokenize and convert_tokens_to_ids separately.

Before we can do that, though, we need to talk about some of BERT's formatting requirements.

## 3.2. Required Formatting

The above code left out a few required formatting steps that we'll look at here.

Side Note: The input format to BERT seems "over-specified" to me... We are required to give it a number of pieces of information which seem redundant, or like they could easily be inferred from the data without us explicity providing it. But it is what it is, and I suspect it will make more sense once I have a deeper understanding of the BERT internals.

We are required to:

1. Add special tokens to the start and end of each sentence.
2. Pad & truncate all sentences to a single constant length.
3. Explicitly differentiate real tokens from padding tokens with the "attention mask".

#### Special tokens
* [SEP] at the end of every sentence
* [CLS] for classification tasks -- the beginning of every sentence.

#### Sentence Length & Attention Mask
The sentences in our dataset obviously have varying lengths, so how does BERT handle this?

BERT has two constraints:

1. All sentences must be padded or truncated to a single, fixed length.
2. The maximum sentence length is 512 tokens.

Padding is done with a special [PAD] token, which is at index 0 in the BERT vocabulary. The below illustration demonstrates padding out to a "MAX_LEN" of 8 tokens.

The "Attention Mask" is simply an array of 1s and 0s indicating which tokens are padding and which aren't (seems kind of redundant, doesn't it?!). This mask tells the "Self-Attention" mechanism in BERT not to incorporate these PAD tokens into its interpretation of the sentence.

The maximum length does impact training and evaluation speed, however. For example, with a Tesla K80:

MAX_LEN = 128 --> Training epochs take ~5:28 each

MAX_LEN = 64 --> Training epochs take ~2:57 each

## 3.3 Tokenize Dataset
The transformers library provides a helpful encode function which will handle most of the parsing and data prep steps for us.

Before we are ready to encode our text, though, we need to decide on a maximum sentence length for padding / truncating to.

The below cell will perform one tokenization pass of the dataset in order to measure the maximum sentence length.

In [17]:
max_len = 0

# For every sentence...
for sent in text_content:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)


Max sentence length:  234


As there are some longer test sentences, I'll set the maximum length to 500.

Now we're ready to perform the real tokenization.

The tokenizer.encode_plus function combines multiple steps for us:

1. Split the sentence into tokens.
2. Add the special [CLS] and [SEP] tokens.
3. Map the tokens to their IDs.
4. Pad or truncate all sentences to the same length.
5. Create the attention masks which explicitly differentiate real tokens from [PAD] tokens.

The first four features are in tokenizer.encode, but I'm using tokenizer.encode_plus to get the fifth item (attention masks). Documentation is [here](https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus).

In [18]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in text_content:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 100,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
token_tensor = torch.cat(input_ids, dim=0)
attention_masks_tensor = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(labels)  # currently, not using labels (won't evaluate it... using just pre-trained model)

# Print sentence 0, now as a list of IDs.
print('Original: ', text_content[2])
print('Token IDs:', token_tensor[2])



Original:   if you would prefer  receive a hard copy of the newsletter in the mail, rather than viewing it on the website, please contact us
Token IDs: tensor([  101,  2065,  2017,  2052,  9544,  4374,  1037,  2524,  6100,  1997,
         1996, 17178,  1999,  1996,  5653,  1010,  2738,  2084, 10523,  2009,
         2006,  1996,  4037,  1010,  3531,  3967,  2149,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])


In [19]:
print (len(attention_masks_tensor)) #73725 sentences for Kate Symes  #42121 sentences for Phillip Love
# after subsetting into sent mails, 18865 sentences for Phillip Love
print (len(token_tensor[2]))

677
100


In [0]:
def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [19]:
segment_ids = get_segments(token_tensor, 100) 

IndexError: ignored

## 3.4. Tokenizing in JSON file to extract attentions (link [here](https://github.com/clarkkev/attention-analysis/blob/master/preprocess_unlabeled.py))

Don't think I will use this method. 
Just use the plain way--and use the config.attention... function to extract attention matrices and summarize!! 


# 4. Extracting Embeddings

## 4.1. Running BERT on our text
Next we need to convert our data to torch tensors and call the BERT model. The BERT PyTorch interface requires that the data be in torch tensors rather than Python lists, so we convert the lists here - this does not change the shape or the data.

In [0]:
#Detaching the emails dataset for the memory space before running BERT
del emails_df 

Calling from_pretrained will fetch the model from the internet. When we load the bert-base-uncased, we see the definition of the model printed in the logging. The model is a deep neural network with 12 layers! Explaining the layers and their functions is outside the scope of this post, and you can skip over this output for now.

model.eval() puts our model in evaluation mode as opposed to training mode. In this case, evaluation mode turns off dropout regularization which is used in training.

In [21]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # whether the model returns all hidden-states.
                                  output_attentions=True,  # whether attention layer is returned
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

Next, let's evaluate BERT on our text in each account, and fetch the hidden states of the network!

> Side note: torch.no_grad tells PyTorch not to construct the compute graph during this forward pass (since we won't be running backprop here)--this just reduces memory consumption and speeds things up a little.

In [0]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(token_tensor)   # currently, ignoring the segment tensor

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]  # this is the hidden state info

In [25]:
hidden_states

(tensor([[[ 1.6855e-01, -2.8577e-01, -3.2613e-01,  ..., -2.7571e-02,
            3.8253e-02,  1.6400e-01],
          [ 2.5102e-01, -1.3555e+00, -5.1271e-01,  ...,  4.3443e-01,
           -5.2151e-01,  5.9567e-01],
          [ 7.7984e-01,  5.3886e-01,  1.9515e-01,  ...,  5.5381e-01,
            1.0014e+00, -8.7890e-01],
          ...,
          [ 5.8332e-01, -1.9910e-01,  8.1386e-01,  ..., -2.4236e-01,
            3.5850e-01,  1.5621e-01],
          [-2.4192e-01,  5.2873e-01,  5.1534e-01,  ...,  3.7666e-02,
            5.9429e-01,  7.3446e-01],
          [-4.4828e-01,  4.6353e-01,  3.9365e-01,  ..., -5.3819e-01,
           -9.1579e-02, -3.9472e-02]],
 
         [[ 1.6855e-01, -2.8577e-01, -3.2613e-01,  ..., -2.7571e-02,
            3.8253e-02,  1.6400e-01],
          [ 5.4667e-01,  3.3008e-01, -9.2265e-01,  ...,  9.1499e-01,
            8.3508e-01, -2.4775e-01],
          [-6.2884e-01,  4.4862e-01,  6.3140e-01,  ...,  5.9740e-01,
            5.3136e-01,  2.4762e-01],
          ...,
    

In [9]:
#%cd /content/drive/My Drive
#!pwd
#import tensorflow as tf
import torch
#torch.save(outputs, '/content/drive/My Drive/outputs_tensor.pt')

outputs = torch.load('/content/drive/My Drive/outputs_tensor.pt')


RuntimeError: ignored

## 3.2. Understanding the Output

The full set of hidden states for this model, stored in the object hidden_states, is a little dizzying. This object has four dimensions, in the following order:

1. The layer number (13 layers)
2. The batch number (1 sentence)
3. The word / token number (22 tokens in our sentence)
4. The hidden unit / feature number (768 features)

Wait, 13 layers? Doesn't BERT only have 12? It's 13 because the first element is the input embeddings, the rest is the outputs of each of BERT's 12 layers.

That’s 219,648 unique values just to represent our one sentence!

The second dimension, the batch size, is used when submitting multiple sentences to the model at once; here, though, we just have one example sentence.

In [42]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 677
Number of tokens: 100
Number of hidden units: 768


Let's take a quick look at the range of values for a given layer and token.

You'll find that the range is fairly similar for all layers and tokens, with the majority of values falling between [-2, 2], and a small smattering of values around -10.

In [43]:
input_ids[0:10]

[tensor([[  101,  4012,  1033,  1024,  5958,  1010,  1051, 27421,  2121,  2539,
           1010,  2184,  1024,  2871,  2572,  1024,  5143,  1010, 20475,  1025,
           3608,  2075,  1010,  2728,  1025, 12075, 16001,  1010,  3660,  1025,
           3016,  3238,  1010,  9802,  1025,  4562,  1010,  6285,  1025,  5041,
           4974,  1010,  6902,  1025,  7641,  1010,  6384,  1025,  2011, 21716,
           1010,  2198,  1025,  5252,  1010,  2728,  1025,  8902,  2140,  1010,
           8057,  1025, 24119,  1010,  2198,  1025,  9574,  1010,  2577,  1025,
          27831,  2140,  1010,  7010,  1025,  2585,  1010,  3487,  1025,  2139,
          23736,  2638,  1010,  4656,  1025, 13109,  5162, 11528,  1010,  2984,
           1025,  2413,  1010, 26568,  1025, 17297,  1010,  3960,  1025,   102]]),
 tensor([[  101,  1000,  2017,  2089,  3193,  1996, 17178,  2011, 29347, 18965,
           2256,  4037,  2012,  7479,   102,     0,     0,     0,     0,     0,
              0,     0,     0,     0,

In [44]:
hidden_states[12]

tensor([[[-8.7071e-01, -1.7167e-01, -4.1474e-01,  ...,  5.0643e-01,
           7.5320e-01,  6.1644e-01],
         [ 1.6066e-01, -1.1465e+00, -1.2744e-01,  ...,  4.9624e-01,
           1.8019e+00,  1.8012e-01],
         [ 2.7691e-01, -6.7028e-01,  8.6098e-01,  ...,  6.8047e-01,
           1.0129e+00, -2.4723e-01],
         ...,
         [ 3.0464e-01, -3.3218e-01, -1.8275e-01,  ...,  4.5554e-01,
           3.0901e-01, -2.0985e-01],
         [-1.2076e+00, -7.0128e-01, -1.1874e+00,  ...,  8.8510e-01,
          -1.3617e-02,  1.5706e-01],
         [ 2.6108e-01,  2.7389e-01, -4.1015e-01,  ...,  2.1230e-01,
          -3.6857e-01, -8.9444e-02]],

        [[-3.7418e-01,  1.9222e-01,  5.1624e-01,  ..., -4.2429e-01,
           5.9706e-01, -6.2933e-01],
         [ 5.2100e-01,  2.4934e-01, -5.4365e-04,  ...,  2.9755e-01,
           7.2869e-02, -2.3605e-01],
         [-7.1197e-01,  8.3246e-02,  8.6361e-01,  ..., -6.6959e-02,
           2.8622e-01, -1.3424e+00],
         ...,
         [ 2.3591e-01, -1

In [45]:
tokenizer.vocab.get("please")  # 1996
outputs.attention_weights

AttributeError: ignored

In [1]:
# For the 5th token in our sentence, select its feature values from layer 5.
token_i = 3  #"view" # 6 and 10 are bank
sentence_i = 1    
#sentence_i = 10
layer_i = 12

vec = hidden_states[layer_i][sentence_i][token_i]

print (token_tensor[token_i]) 
print (text_content[sentence_i]) 

#indexed_tokens = tokenizer.convert_tokens_to_ids(token_tensor)
tokenized_text = tokenizer.tokenize(text_content[sentence_i])
print (tokenized_text[token_i]) 

# Display the words with their indeces.
#for tup in zip(text_content, indexed_tokens):
#    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

# Plot the values as a histogram to show their distribution.
plt.figure(figsize=(10,10))
plt.hist(vec, bins=200)
plt.show()  # this visualizes the vector for token "view" in the last hidden layer

NameError: ignored

In [79]:
#  Let's see if "view" has multiple tokens

for i, token_str in enumerate(tokenized_text):
  print (i, token_str)


  

0 "
1 you
2 may
3 view
4 the
5 newsletter
6 by
7 ae
8 ##ssing
9 our
10 website
11 at
12 www


Grouping the values by layer makes sense for the model, but for our purposes we want it grouped by token.

Current dimensions:

[# layers, # batches, # tokens, # features]

Desired dimensions:

[# tokens, # layers, # features]

Luckily, PyTorch includes the permute function for easily rearranging the dimensions of a tensor.

However, the first dimension is currently a Python list!

In [0]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 22, 768])


Let's combine the layers to make this one whole big tensor.

In [0]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 22, 768])

Let's get rid of the "batches" dimension since we don't need it.

In [0]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 22, 768])

Finally, we can switch around the "layers" and "tokens" dimensions with permute.

In [0]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([22, 13, 768])

##3.3. Creating word and sentence vectors from hidden states

### Word Vectors

Now, what do we do with these hidden states? We would like to get individual vectors for each of our tokens, or perhaps a single vector representation of the whole sentence, but for each token of our input we have 13 separate vectors each of length 768.

In order to get the individual vectors we will need to combine some of the layer vectors...but which layer or combination of layers provides the best representation?

Unfortunately, there's no single easy answer... Let's try a couple reasonable approaches, though. Afterwards, I'll point you to some helpful resources which look into this question further.

Word Vectors
To give you some examples, let's create word vectors two ways.

First, let's **concatenate** the last four layers, giving us a single word vector per token. Each vector will have length 4 x 768 = 3,072.

In [0]:
# Stores the token vectors, with shape [22 x 3,072]
token_vecs_cat = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:
    
    # `token` is a [12 x 768] tensor

    # Concatenate the vectors (that is, append them together) from the last 
    # four layers.
    # Each layer vector is 768 values, so `cat_vec` is length 3,072.
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Shape is: 22 x 3072


As an alternative method, let's try creating the word vectors by **summing** together the last four layers.

In [0]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 22 x 768


### Sentence Vectors


To get a single vector for our entire sentence we have multiple application-dependent strategies, but a simple approach is to average the second to last hiden layer of each token producing a single 768 length vector.

In [0]:
# `hidden_states` has shape [13 x 1 x 22 x 768]

# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [0]:
print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


##3.4. Confirming contextually dependent vectors

To confirm that the value of these vectors are in fact contextually dependent, let's look at the different instances of the word "bank" in our example sentence:

"After stealing money from the **bank vault**, the **bank robber** was seen fishing on the **Mississippi river bank**."

Let's find the index of those three instances of the word "bank" in the example sentence.

In [0]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 after
2 stealing
3 money
4 from
5 the
6 bank
7 vault
8 ,
9 the
10 bank
11 robber
12 was
13 seen
14 fishing
15 on
16 the
17 mississippi
18 river
19 bank
20 .
21 [SEP]


They are at 6, 10, and 19.

For this analysis, we'll use the word vectors that we created by summing the last four layers.

We can try printing out their vectors to compare them.

In [0]:
print('First 5 vector values for each instance of "bank".')
print('')
print("bank vault   ", str(token_vecs_sum[6][:5]))
print("bank robber  ", str(token_vecs_sum[10][:5]))
print("river bank   ", str(token_vecs_sum[19][:5]))

First 5 vector values for each instance of "bank".

bank vault    tensor([ 3.3596, -2.9805, -1.5421,  0.7065,  2.0031])
bank robber   tensor([ 2.7359, -2.5577, -1.3094,  0.6797,  1.6633])
river bank    tensor([ 1.5266, -0.8895, -0.5152, -0.9298,  2.8334])


We can see that the values differ, but let's calculate the cosine similarity between the vectors to make a more precise comparison.



In [0]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for  *similar*  meanings:  0.94
Vector similarity for *different* meanings:  0.69
